In [167]:
import time
import datetime
import os
import yaml
import math
import sys
import gurobipy



import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

print(f"python version: {sys.version}")
print(f"gurobipy version: {gurobipy.gurobi.version()}")


python version: 3.9.6 (default, Sep 26 2022, 11:37:49) 
[Clang 14.0.0 (clang-1400.0.29.202)]
gurobipy version: (11, 0, 0)


In [168]:
# 添加相對路徑到 sys.path
sys.path.append('../')  
from path_config import INSTANCES_DIR
def load_specific_yaml(filename):
    """
    加載指定的 YAML 檔案。
    
    Parameters:
    filename (str): 在 instances 資料夾中的 YAML 檔案名。
    
    Returns:
    dict: YAML 檔案內容。
    """
    file_path = os.path.join(INSTANCES_DIR, filename)
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

# Load experiments yaml
config = load_specific_yaml('instance_F_4.yaml')
config['lambda_for_G'] = 3e-5
config['M'] = 1000000
config

{'A_opponent_bar': [11977, 13657, 14483, 10742, 1901, 2349],
 'B': [[2, 1, 3],
  [3, 4, 3],
  [4, 3, 1],
  [4, 2, 4],
  [3, 1, 2],
  [4, 3, 4],
  [3, 4, 2],
  [1, 4, 2],
  [1, 1, 1],
  [1, 3, 1],
  [4, 3, 1],
  [2, 3, 4],
  [4, 1, 1],
  [3, 3, 2],
  [4, 4, 3],
  [2, 1, 1],
  [2, 2, 2],
  [3, 3, 1],
  [3, 3, 4],
  [4, 3, 2]],
 'C': [0.00011616207457673666,
  8.88895345516699e-05,
  0.00020126776776734792,
  0.00017008510941133147,
  0.00020708295371651685,
  0.00016890655140340422,
  0.0001632270551350362,
  0.00018247590411572795,
  0.00020352648524369984,
  0.00018690992185115332,
  0.0001787888461543107,
  8.361047873315134e-05,
  0.00018746767148598167,
  8.809406796858187e-05,
  0.00022458004302614433,
  0.0001996058970187463,
  5.2544531863756864e-05,
  0.00018600741540593155,
  0.0001907928878168022,
  8.574734193245077e-05],
 'D': [[95.21,
   42.38,
   22.02,
   89.64,
   49.66,
   65.05,
   75.77,
   72.9,
   91.41,
   92.89,
   42.19,
   90.21,
   27.2,
   21.1,
   81.06,
   1

In [169]:
I = set(range(0, config['i_amount']))  
J = set(range(0, config['j_amount']))  
K = set(range(0, config['k_amount']))  
L = set(range(0, config['l_amount']))  
'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
u : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
===============================
'''

'\nDecision Variables\n===============================\ny : Whether location j build facility or not, Dim: (j) , BINARY\nx : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS\naX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS\n===============================\n\nOther Variables\nu : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS\nP : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS\nTA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS\n===============================\n'

In [170]:
# # Model initial (G, no reformulate)
# model = Model("Competitive Facility Location")
# model.setParam('NumericFocus', 3)
# y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
# x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
# aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

# u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
# P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, utility vs TA ratio", lb=0.0, ub=1.0) # Probability of i choosing j
# TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
# TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
# G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5

# model.update()

# model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
# model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
# model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
# model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

# model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
# model.addConstrs((u[i, j] == P[i, j] * TA[i] for i in I for j in J), "(3.8) calculate P by uij/TAi, 分母要是常數所以用乘的")
# ## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
# model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

# ## G的計算
# # def G(tai):
# #     return 1 - np.exp(-config['lambda_for_G'] * tai)
# model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# # Gurobi無法直接在目標式有exponential
# for i in I:
#     model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# # Objective function: Maximize profit by attracting customers - costs
# objective = ((quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (quicksum(P[i, j] for j in J)) for i in I)) \
#           - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J)))
# model.setObjective(objective, GRB.MAXIMIZE)


# # Solve and Output
# model.optimize()
# if model.status == GRB.OPTIMAL:
#     print("Optimal solution found.")
# else:
#     print("No optimal solution found.")
#     model.computeIIS()
#     # 将IIS输出到文件
#     # model.write("model_iis.mps")
#     print("IIS written to file 'model_iis.mps'")



In [171]:
# Model  (G, reformulate)
model = Model("Competitive Facility Location")
model.setParam('NumericFocus', 3)
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, opponent vs TA ratio", lb=0.0, ub=1.0) # 改這裡
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5

model.update()

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((u[i, j] == ((quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i")
## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

model.addConstrs(((quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) \
                  == P[i, j] * TA[i] for i in I for j in J), "A_opponent對比TAI, 分母要是常數所以用乘的")

## G的計算
# def G(tai):
#     return 1 - np.exp(-config['lambda_for_G'] * tai)
model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# Objective function: Maximize profit by attracting customers - costs
objective = (quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (1 - quicksum(P[i, j] for j in J)) for i in I)) \
          - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J))
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    model.computeIIS()
    # 将IIS输出到文件
    # model.write("model_iis.mps")
    print("IIS written to file 'model_iis.mps'")



Set parameter NumericFocus to value 3
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 21.5.0 21F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 213 rows, 315 columns and 895 nonzeros
Model fingerprint: 0x6b769312
Model has 100 quadratic objective terms
Model has 100 quadratic constraints
Model has 5 general constraints
Variable types: 295 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [5e-05, 3e+03]
  QObjective range [2e+03, 6e+03]
  Bounds range     [1e-06, 1e+05]
  RHS range        [5e+00, 9e+01]
  QRHS range       [2e+01, 9e+01]
Presolve added 0 rows and 55 columns
Presolve removed 72 rows and 0 columns
Presolve time: 0.00s
Presolved: 742 rows, 472 columns, 1927 nonzeros
Presolved model has 5 SOS constraint(s)
Presolved model has 200 bilinear constraint(s

     0     1 5596.87605    0  200 5536.27918 5596.87605  1.09%     -    0s
H   46    20                    5536.3336182 5557.72375  0.39%  53.6    0s
H   49    20                    5536.8218140 5557.72375  0.38%  51.6    0s

Explored 135 nodes (5791 simplex iterations) in 0.58 seconds (0.55 work units)
Thread count was 8 (of 8 available processors)

Solution count 4: 5536.82 5536.33 5536.28 -257.305 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.536821814001e+03, best bound 5.536821852189e+03, gap 0.0000%
Optimal solution found.


In [172]:
# Model  (G, reformulate, add E)
# 直接加入e進去utility
model = Model("Competitive Facility Location")
model.setParam('NumericFocus', 3)
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="utility(u)", lb=0) # utility var(3.5)
P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="P, opponent vs TA ratio", lb=0.0, ub=1.0) # 改這裡
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA", lb=0.00000000)
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="negative TAi*lambda(e's power)", lb= -100000, ub= -0.000001) #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars", lb=0.0, ub=1) #e^-(lambda*TAi), ub=0.5

place_utility = model.addVars(J, vtype=GRB.CONTINUOUS, name="place_utility", lb=0)

model.update()

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((place_utility[j] == quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j] for j in J), "place utility before e")


model.addConstrs((u[i, j] == ( (-0.0000007*place_utility[j]**2 + 1.1*place_utility[j]) \
                              / (config['D'][i][j]**2)) for i in I for j in J), "(3.5) utility for facility j to customer i加入e以後")

## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + (quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) for i in I), "(3.7) calculate total TA by u and A_bar")

model.addConstrs(((quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L)) \
                  == P[i, j] * TA[i] for i in I for j in J), "A_opponent對比TAI, 分母要是常數所以用乘的")

## G的計算
# def G(tai):
#     return 1 - np.exp(-config['lambda_for_G'] * tai)
model.addConstrs((TAi_lambda[i] == (-config['lambda_for_G'] * TA[i]) for i in I), "negative tai*lambda constr")
# Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExpA(TAi_lambda[i], G_exp_vars[i], math.e, name=f"G_exp_constr_{i}")



# Objective function: Maximize profit by attracting customers - costs
objective = (quicksum(config['H'][i] * (1 - G_exp_vars[i]) * (1 - quicksum(P[i, j] for j in J)) for i in I)) \
          - (quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J))
model.setObjective(objective, GRB.MAXIMIZE)


# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")
    model.computeIIS()
    # 将IIS输出到文件
    # model.write("model_iis.mps")
    # print("IIS written to file 'model_iis.mps'")



In [173]:
for var in model.getVars():
    print(f"{var.varName} = {var.x}")
print("")

y[0] = 0.0
y[1] = 1.0
y[2] = 1.0
y[3] = 0.0
y[4] = 0.0
y[5] = 0.0
y[6] = 1.0
y[7] = 0.0
y[8] = 0.0
y[9] = 0.0
y[10] = 1.0
y[11] = 0.0
y[12] = 1.0
y[13] = 1.0
y[14] = 0.0
y[15] = 0.0
y[16] = 0.0
y[17] = 0.0
y[18] = 0.0
y[19] = 1.0
x[0,0] = 0.0
x[0,1] = 0.0
x[0,2] = 0.0
x[1,0] = 4.0
x[1,1] = 0.0
x[1,2] = 0.0
x[2,0] = 4.0
x[2,1] = 0.0
x[2,2] = 1.0
x[3,0] = 0.0
x[3,1] = 0.0
x[3,2] = 0.0
x[4,0] = 0.0
x[4,1] = 0.0
x[4,2] = 0.0
x[5,0] = 0.0
x[5,1] = 0.0
x[5,2] = 0.0
x[6,0] = 2.0
x[6,1] = 3.0
x[6,2] = 3.0
x[7,0] = 0.0
x[7,1] = 0.0
x[7,2] = 0.0
x[8,0] = 0.0
x[8,1] = 0.0
x[8,2] = 0.0
x[9,0] = 0.0
x[9,1] = 0.0
x[9,2] = 0.0
x[10,0] = 0.0
x[10,1] = 3.0
x[10,2] = 0.0
x[11,0] = 0.0
x[11,1] = 0.0
x[11,2] = 0.0
x[12,0] = 0.0
x[12,1] = 4.0
x[12,2] = 3.0
x[13,0] = 4.0
x[13,1] = 3.0
x[13,2] = 2.0
x[14,0] = 0.0
x[14,1] = 0.0
x[14,2] = 0.0
x[15,0] = 0.0
x[15,1] = 0.0
x[15,2] = 0.0
x[16,0] = 0.0
x[16,1] = 0.0
x[16,2] = 0.0
x[17,0] = 0.0
x[17,1] = 0.0
x[17,2] = 0.0
x[18,0] = 0.0
x[18,1] = 0.0
x[18,2] = 0.0
x[

# References

## G calculation substitude exponential

https://www.gurobi.com/documentation/10.0/refman/py_model_agc_exp.html

https://support.gurobi.com/hc/en-us/community/posts/360077178491-Build-an-objective-function-with-Log-and-Exponential-inside-

https://support.gurobi.com/hc/en-us/community/posts/10481250723217-How-to-add-log-and-exponential-term-in-objective-function
